In [1]:
import torch
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer,AutoModelForCausalLM,pipeline
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

c:\KM\Virtual ENV\venv2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
loader=PyMuPDFLoader('data/Private_Cred_Fin_Stab_Paper.pdf')
data=loader.load()
data[0].metadata

{'producer': 'pdfTeX-1.40.25',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2025-12-16T18:54:15+00:00',
 'source': 'data/Private_Cred_Fin_Stab_Paper.pdf',
 'file_path': 'data/Private_Cred_Fin_Stab_Paper.pdf',
 'total_pages': 56,
 'format': 'PDF 1.5',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'moddate': '2025-12-16T18:54:15+00:00',
 'trapped': '',
 'modDate': 'D:20251216185415Z',
 'creationDate': 'D:20251216185415Z',
 'page': 0}

In [3]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1200,chunk_overlap=200,separators=['\n\n','\n','.',' '])
chunks=text_splitter.split_documents(data)
len(chunks)

143

In [4]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
vector_db=FAISS.from_documents(chunks,embeddings)
vector_db.save_local('RAG INDEX')

In [5]:
MODEL='Qwen/Qwen2.5-1.5B-Instruct'
tokenizer=AutoTokenizer.from_pretrained(MODEL)
model=AutoModelForCausalLM.from_pretrained(MODEL,device_map='auto',dtype=torch.bfloat16,low_cpu_mem_usage=True)
pipe=pipeline(task='text-generation',temperature=0.2,do_sample=True,tokenizer=tokenizer,model=model,max_new_tokens=512,repetition_penalty=1.2,
              no_repeat_ngram_size=3)

Device set to use cuda:0


In [6]:
def format_prompt(question):
    search_resluts=vector_db.similarity_search(question,k=5)
    context='\n---\n'.join([doc.page_content for doc in search_resluts])
    prompt=f'''You are a Financial Analyst.Answer the question only from the context provided.
    RULES:
    1.You must be factually Accurate
    2.If the information is not sufficient then mention 'Not enough information is provided in the document'
    3.You must not answer beyond the context provided
    4.Cite the financial metrics and definitions exactly as present in the context
context:
{context}

question:{question}
Answer: '''
    response=pipe(prompt,return_full_text=False)
    return response[0]['generated_text']

In [7]:
print(format_prompt('Summarize this Document'))

 The given text discusses various aspects related to bond investment companies' (BDC's) leverage reduction strategies across different categories based on their Asset Coverage Ratio (ACR): publically traded, perpetually non-tradable, term non-trading, and those classified under "perpetual" category.

The key points summarized:

- **Leverage Reduction Strategies**: 
   - Assets Sales: Companies sell off excess assets to reduce overall liabilities.
   - Use Cash Buffers to Repay Debt: Utilizing existing funds within the company to pay down debts directly.
   -
   -

- **Impact Analysis**:
   - Across these groups, there was significant variation in how much they reduced their leverages; notably, lower ACRs led to more substantial reductions compared to higher ones.
   
- **Risk Metrics**:
     - Industry Default Beta captures the volatility associated with potential defaults among investments held by the BCDs.
     
- **Market Conditions**:
      - In severe adverse market scenarios, cer

In [8]:
print(format_prompt('What is BDC and also What are BDC assets?'))

 Based on the given context:

BDC stands for "Business Development Companies" - These are specialized types of mutual funds designed specifically for investing in smaller businesses.

The text states that BDC's assets include both principal amounts invested in various forms like loans, cash reserves, etc., along with any other relevant securities held within them. 

Specifically mentioned data points show how much each type contributes towards overall BDC asset figures; e.g.,
* Loans represent around 87% of most BDC portfolios' composition;
* Equity represents approximately 5.9%;
* Other components may vary based on specific details per individual BDC;

So while we don't get exact numbers here, this gives us insight into typical proportions used in calculating BDC-related assets.


In [9]:
print(format_prompt('What information does the summary statistics table of 2024Q4 provide?'))

2


In [10]:
print(format_prompt('What are the portfolio holdings of PitchBook'))

 Portfolio holdings data includes details like issuer name,industry,fair value,amortised cost,number of shares,maturity dates,and terms related to loans.It covers both publicly listed bdc's and jvs.

The above text describes what pitchbook does regarding obtaining portfolio holdings info for bdc companies.It doesn't specify if this applies specifically to them getting detailed info on specific investments made within those holdings. 

So based solely off the given passage we can say:

PitchBook obtains portfolio holdings information through several means - 
- From pitching book itself where there may be detailed records available for individual investment items.
- Through third party databases/sources when directly accessing isn’t possible due to restrictions/privacy concerns etc..
However no explicit statement says "pitchbook gets exact/portfolio level holdings" so cannot confirm fully unless explicitly stated otherwise. 

Therefore I would need further clarification before stating de

In [11]:
print(format_prompt('What is DRIP?What Role does DRIP play in this document?'))

 Dividend Reinvest Plan (DRP), or simply "dividends," refers to the practice whereby a company allows its shareholders to automatically purchase additional shares based upon the amount of dividends received during the previous period. This process can help retain shareholder wealth over time because it enables them to benefit from any future price appreciation.

In the given text, DRIPS play a significant role in analyzing how changes in dividend policies affect the overall leverage levels within various categories of business development corporations (BDC)s. Specifically:

- **Impact Analysis**: It's mentioned that there exists a relationship between the Distribution ReInvestment Rate (DRI%) - defined as the ratio of the value generated through dividend re-investment programs relative to total dividends distributed per share held - and the level of debt reduction among different types (perpetual vs term/non-traded) of BCDs. 

- **Model Application**: The study uses two models – one wi

In [12]:
print(format_prompt('What is RIC? How much net income does BDC distribute?'))

1.RIC stands for "Registered Investment Company". It refers to a type of company formed under federal law that allows individuals to invest money through shares without having direct ownership rights over the underlying property.

2.Based on the given text, BCD distributes 9/10th of its net income as regular dividends. Specifically:

   * For perpetuity BDC's, this distribution requirement translates to paying out approximately **$8** billion annually ($8 / year).
   
   * Permanently invested BDC’s also need to meet this payout rule; however, since there isn't specific numerical figures mentioned here regarding how much each permanent fund contributes towards annual payouts, I cannot give you precise numbers about individual amounts distributed among those categories. 

In summary, while both types of BDs adhere strictly to distributing 9 times their net earnings as dividends according to IRS regulations, the exact amount varies depending upon the size and nature of each particular BD